In [83]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import Dense, Dropout, InputLayer, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.regularizers import l2
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
import joblib

In [84]:
#Define the coati optimizer
class Coati(tf.keras.optimizers.Optimizer):
    def __init__(self, learning_rate=0.01, momentum=0.9, name='Coati', **kwargs):
        """Constructs a new Coati optimizer.

        Args:
            learning_rate: A Tensor or a floating point value. The learning rate.
            momentum: A floating point value. The momentum for the optimizer.
            name: Optional name prefix for the operations created when applying gradients.
            **kwargs: Keyword arguments. Allowed to be one of "clipnorm" or "clipvalue".
        """
        super(Coati, self).__init__(name, **kwargs)
        self._set_hyper('learning_rate', kwargs.get('lr', learning_rate))  # handle lr=learning_rate
        self._set_hyper('momentum', momentum)
    def _create_slots(self, var_list):
        for var in var_list:
            self.add_slot(var, 'momentum')

    def _resource_apply_dense(self, grad, var, apply_state=None):
        lr_t = self._decayed_lr(var.dtype.base_dtype)
        momentum_var = self.get_slot(var, 'momentum')
        momentum_var_t = tf.multiply(momentum_var, self._get_hyper('momentum'))
        momentum_var_t = momentum_var_t + grad
        var.assign_sub(lr_t * momentum_var_t)

    def _resource_apply_sparse(self, grad, var, indices, apply_state=None):
        lr_t = self._decayed_lr(var.dtype.base_dtype)
        momentum_var = self.get_slot(var, 'momentum')
        momentum_var_t = tf.multiply(momentum_var, self._get_hyper('momentum'))
        momentum_var_t = momentum_var_t + tf.gather(grad, indices)
        var.assign_sub(lr_t * momentum_var_t)

    def get_config(self):
        config = super(Coati, self).get_config()
        config.update({
            'learning_rate': self._serialize_hyperparameter('learning_rate'),
            'momentum': self._serialize_hyperparameter('momentum'),
        })
        return config


In [85]:
data = pd.read_csv('C:/Users/Akoba/Desktop/START up/Covid19_research_project/data/raw/vitals.csv')

In [86]:
# Perform basic data exploration (optional)
print(data.head())

   ID  Oxygen  PulseRate  Temperature    Result
0   0      98         65           95  Negative
1   1      96         92           95  Negative
2   2      95         92           99  Negative
3   3      97         56           96  Negative
4   4      88         94           98  Positive


In [87]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID           10000 non-null  int64 
 1   Oxygen       10000 non-null  int64 
 2   PulseRate    10000 non-null  int64 
 3   Temperature  10000 non-null  int64 
 4   Result       10000 non-null  object
dtypes: int64(4), object(1)
memory usage: 390.8+ KB
None


In [88]:

print(data.describe())

                ID        Oxygen     PulseRate   Temperature
count  10000.00000  10000.000000  10000.000000  10000.000000
mean    4999.50000     92.548900     84.976600    100.000700
std     2886.89568      4.611197     26.305841      3.185045
min        0.00000     85.000000     40.000000     95.000000
25%     2499.75000     88.000000     63.000000     97.000000
50%     4999.50000     93.000000     85.000000    100.000000
75%     7499.25000     97.000000    108.000000    103.000000
max     9999.00000    100.000000    130.000000    105.000000


In [89]:
print(data['Result'].unique())

['Negative' 'Positive']


In [90]:
data['Result'] = data['Result'].map({'Negative': 0, 'Positive': 1})

In [91]:
# Separate features and labels
X = data.drop('Result', axis=1)  #  'Result' is the target column
y = data['Result']

In [92]:
print(data.Result)

0       0
1       0
2       0
3       0
4       1
       ..
9995    0
9996    1
9997    0
9998    1
9999    1
Name: Result, Length: 10000, dtype: int64


In [93]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [94]:
# Standardize the features
import numpy as np
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
np.isnan(X_train).any()
np.isinf(X_train).any()

False

In [95]:
# Convert to TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(32)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(32)

In [96]:
model = Sequential([
    InputLayer(input_shape=(X_train.shape[1],)),  # Input layer matches feature size

    # Hidden Layers (optimized)
    Dense(64, activation='relu', kernel_regularizer=l2(0.001)), # Smaller initial layer, L2 regularization
    BatchNormalization(),
    Dropout(0.4),  
    Dense(32, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.4),

    # Output Layer (sigmoid for binary classification)
    Dense(1, activation='sigmoid') 
])

c:\Users\Akoba\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\input_layer.py:25: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


In [97]:
# Compile the model with the Coati optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [98]:
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_15 (Dense)                │ (None, 64)             │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_10          │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_11          │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,817 (11.00 KB)

 Trainable params: 2,625 (10.25 KB)

 Non-trainable params: 192 (768.00 B)

In [99]:
# Train the model
history = model.fit(train_dataset, epochs=20, validation_data=test_dataset)

Epoch 1/20


c:\Users\Akoba\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\losses\losses.py:22: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)


250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7791 - loss: 0.0500 - val_accuracy: 0.9095 - val_loss: 0.0499
Epoch 2/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8967 - loss: 0.0497 - val_accuracy: 0.9180 - val_loss: 0.0493
Epoch 3/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9062 - loss: 0.0491 - val_accuracy: 0.9125 - val_loss: 0.0488
Epoch 4/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8993 - loss: 0.0487 - val_accuracy: 0.9140 - val_loss: 0.0484
Epoch 5/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8909 - loss: 0.0483 - val_accuracy: 0.9050 - val_loss: 0.0483
Epoch 6/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8903 - loss: 0.0481 - val_accuracy: 0.9020 - val_loss: 0.0484
Epoch 7/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8816 - loss: 0.0483 - val_accuracy: 0.9050 - val_loss: 0.0483
Epoch 8/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8717 - loss: 0.0484 - val_accuracy: 0.9000 - val_

In [101]:
y_pred = model.predict(X_test)
y_pred_classes = (y_pred > 0.5).astype("int32")
y_true_classes = y_test

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


In [102]:
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, roc_auc_score, roc_curve, auc
import matplotlib.pyplot as plt

In [103]:
accuracy = accuracy_score(y_true_classes, y_pred_classes)
f1 = f1_score(y_true_classes, y_pred_classes, average='weighted')
recall = recall_score(y_true_classes, y_pred_classes, average='weighted')
precision = precision_score(y_true_classes, y_pred_classes, average='weighted')
conf_matrix = confusion_matrix(y_true_classes, y_pred_classes)
roc_auc = roc_auc_score(y_true_classes, y_pred)

In [104]:
# Save the trained model
model.save('C:/Users/Akoba/Desktop/START up/Covid19_research_project/models/mlp/optimized_mlp_model.h5')

In [105]:
# ROC curve
fpr, tpr, thresholds = roc_curve(y_true_classes, y_pred)
roc_auc_value = auc(fpr, tpr)

In [106]:
# Save results to file
with open('C:/Users/Akoba/Desktop/START up/Covid19_research_project/results/evaluation/optimized_mlp_results.txt', 'w') as f:
    f.write('Confusion Matrix:\n')
    f.write(str(conf_matrix))
    f.write('\n\nClassification Report:\n')
    f.write(f'Accuracy: {accuracy:.4f}\n')
    f.write(f'F1 Score: {f1:.4f}\n')
    f.write(f'Recall: {recall:.4f}\n')
    f.write(f'Precision: {precision:.4f}\n')
    f.write(f'ROC AUC: {roc_auc_value:.4f}\n')